# RecommendData

The aim of this notebook is to analyse and transform the data from [zeeguu](https://github.com/zeeguu/data-releases.git) so that it can be used by the recommendation model.

*First, let's prepare our notebook by importing the necessary libraries and defining the paths to the data.*

In [ ]:
# ---------------------------- PREPARING NOTEBOOK ---------------------------- #
# Autoreload
%load_ext autoreload
%autoreload 2

%cd ..

# Random seed
import numpy as np
np.random.seed(42)

# External modules
import os
import mysql
import pandas as pd
from IPython.display import display, Markdown
import time
from tqdm import notebook as tqdm
import sys

# Define PWD as the current git repository
import git
repo = git.Repo('.', search_parent_directories=True)
pwd = repo.working_dir
os.chdir(pwd)

# Internal modules
sys.path.append(pwd)
from src.DataManager import DataManager

# Set global log level
import logging
logging.basicConfig(level=logging.INFO)

## 1. Download the data

Now it's time for us to download the data from **Zeeguu** 😀

⚠️ **Please note! *Please note that the next two parts of this notebook are particularly long and costly in terms of resources and storage, as they will download the databases, run them on the MySQL server (**which you must have installed**) and finally download them in csv format.* To avoid this waiting time, you can run the next cell, which will download the files already extracted for you. You can then go straight on to part 3.

In [ ]:
# --------------------- DOWNLOADING CSVS FROM HUGGINGFACE -------------------- #
# Imports
from huggingface_hub import snapshot_download

# Create folder if not exists
path = os.path.join(pwd, "data", "processed")
if not os.path.exists(path):
    os.makedirs(path)

# Download CSVs
snapshot_download(
    repo_id="OloriBern/FLDE",
    allow_patterns=["recommendation/*.csv"],
    local_dir=path,
    revision="main",
    repo_type="dataset",
)

Otherwise, you can always redo everything manually 🙃

In [ ]:
# ------------------------------- DOWNLOAD DATA ------------------------------ #
data_manager = DataManager()
data_manager.download("recommendation")

We're now going to send the downloaded files to the zeeguu database in **MySQL**. Make sure you have installed **MySQL** and configured the user:
- `user` : zeeguu
- password : zeeguu

Finally, launch the database with the following command:
``bash
sudo /etc/init.d/mysql restart
```

In [ ]:
# ---------------------------- PUSH DATA TO MYSQL ---------------------------- #
data_manager.push_to_mysql()

## 2. Transform the data into dataframes

Now that our data is in the **MySQL** database, we can retrieve it and transform it into dataframes for local storage.

**Please note**: Give your *SQL* server time to load the data. You should have *50 tables* in your database.

In [ ]:
# -------------------------------- LIST TABLES ------------------------------- #
# Définition des identifiants de connection
host = "127.0.0.1"
database = "zeeguu"
user = "zeeguu"
password = "zeeguu"

# Connection à la base de données
db_connection = mysql.connector.connect(
    host=host, user=user, password=password, database=database
)
cursor = db_connection.cursor()

# Liste des tables
tables = []
display(Markdown("*Waiting for tables to be created...*"))
while len(tables) < 50:
    cursor.execute("SHOW TABLES")
    tables = pd.DataFrame(cursor.fetchall()).iloc[:, 0]
    display("{}/50 tables created".format(len(tables)))
    print("", end="\r")
    time.sleep(1)

display(Markdown("### Some tables in the database"))
display(Markdown("There are {} tables in the database".format(len(tables))))
display(Markdown("Here are some of them:"))
display(tables.sample(5))

In [ ]:
# --------------------------- TRANSFORM DATA TO CSV -------------------------- #
# Création du dossier de sortie
output_folder = os.path.join(pwd, "data", "processed", "recommendation")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Exportation des tables
num_empty_tables = 0
for table in tables:
    cursor.execute("SELECT * FROM {}".format(table))
    value = cursor.fetchall()
    if len(value) == 0:
        display(Markdown("- Table **{}** is empty -> **Skipping...**".format(table)))
        num_empty_tables += 1
        continue
    df = pd.DataFrame(value)
    df.columns = [i[0] for i in cursor.description]
    df.to_csv(os.path.join(output_folder, "{}.csv".format(table)), index=False)
    display(Markdown("#### Exported table **{}**".format(table)))
    columns = ", ".join(df.columns)
    count = len(df)
    display(df.describe().loc[["min", "max"]].T)
    display(Markdown("- **Number of rows**: {}".format(count)))

# Give some information about the export
display(
    Markdown(
        "**Exportation completed !** (*{}%* of the tables were empty)".format(
            round(num_empty_tables / len(tables) * 100, 2)
        )
    )
)

# Close connection
cursor.close()
db_connection.close()

## 3. Transform to Surprise format

Now that we have the data available in the correct format, we'll transform it into a format that can be used by the **Surprise** library.
We therefore wish to have a dataframe of the form :

| user_id | item_id | rating | timestamp |
|---------|---------|--------|-----------|

### The rating issue

Users don't actually assign a score to the article they're reading. So we need to find a way of assessing the relevance of an article to a user. We're going to explore several avenues:

#### The reader's estimation of difficulty

We have a table containing the difficulties estimated by users for each item. Assuming that :
- A **1** is too simple,
- A **5** is too difficult,
- A **3** is the ideal difficulty


We can calculate binary ratings: *2 for correctly estimated difficulty, 1 for wrongly estimated difficulty and 0 for no estimation*.


In [ ]:
# -------------------------- CREATE SUPRISE DATASET -------------------------- #
# Importation des données
df = pd.read_csv(
    os.path.join(
        pwd,
        "data",
        "processed",
        "recommendation",
        "article_difficulty_feedback.csv",
    ),
    index_col=0,
)

# Mise en forme des données
df = df[["user_id", "article_id", "difficulty_feedback", "date"]]
df["date"] = pd.to_datetime(df["date"]).astype(int) // 10**9
df.sort_values(by="date", inplace=True)
df.columns = ["user_id", "article_id", "rating", "timestamp"]

# Create ratings
df["rating"] = (abs(df["rating"] - 3) == 0).astype(int)
df["rating"] = df["rating"].replace(2, 1).replace(0, 2)

df["rating"][df["rating"] == 2] = 1
df["rating"][df["rating"] == 0] = 2

# Ajout des 0 pour les utilisateurs qui n'ont pas noté d'article
users = df["user_id"].unique()
articles = df["article_id"].unique()
all_users = np.repeat(users, len(articles))
all_articles = np.tile(articles, len(users))
all_ratings = np.zeros(len(all_users))
all_timestamps = np.zeros(len(all_users))
df_all = pd.DataFrame(
    {
        "user_id": all_users,
        "article_id": all_articles,
        "rating": all_ratings,
        "timestamp": all_timestamps,
    }
)
df = pd.concat([df, df_all], axis=0)

# Display and save
## Create output folder
output_folder = os.path.join(
    pwd,
    "results",
    "recommendation_datasets",
)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
## Save to csv
df.to_csv(os.path.join(output_folder, "dataset_1.csv"), index=False, sep="\t")
## Display
display(Markdown("#### Dataset 1 : *Based on difficulty feedback*"))
display(df.head())
## Display some information
display(Markdown("- **Number of users**: {}".format(len(df["user_id"].unique()))))
display(Markdown("- **Number of articles**: {}".format(len(df["article_id"].unique()))))
display(
    Markdown(
        "- **Percentage of positive ratings**: {}%".format(
            round(df["rating"] > 0).mean() * 100, 2
        )
    )
)

#### The reading time

Now we can use another piece of information provided by our dataset: the time it takes each user to read an article.

*⚠️ Please note that the dataset creators themselves have stated that this information is unreliable!*

---
We'll proceed as follows:
- Sum the time spent by each user on each article,
- Replace abnormal reading times with minimum or maximum estimated reading times,
- Normalize reading times for each user,
- Calculate scores based on normalized reading times.

In [ ]:
# -------------------------- CREATE SURPRISE DATASET ------------------------- #

# Importation des données
df = pd.read_csv(
    os.path.join(
        pwd,
        "data",
        "processed",
        "recommendation",
        "user_reading_session.csv",
    ),
    index_col=0,
)
articles_df = pd.read_csv(
    os.path.join(
        os.getcwd(),
        "Lingorank_LLM" if not (os.getcwd().endswith("Lingorank_LLM")) else "",
        "results",
        "zeeguu_csvs",
        "recommendation",
        "article.csv",
    ),
    index_col=0,
)

# Mise en forme des données
df = df[["user_id", "article_id", "duration", "start_time"]]
df.rename(columns={"start_time": "timestamp"}, inplace=True)
df["timestamp"] = pd.to_datetime(df["timestamp"]).astype(int) // 10**9
df = df.merge(articles_df["word_count"], left_on="article_id", right_index=True)
df = df[df["word_count"].str.isnumeric().astype(bool)].dropna(subset=["word_count"])
df["word_count"] = df["word_count"].astype(int)
df["estimated_reading_time"] = df["word_count"] / 150  # 150 words per minute
df["duration"] = df["duration"] / 60e3
df.drop(columns=["word_count"], inplace=True)
df["article_id"] = df["article_id"].astype(int)

# Somme des temps de lecture par utilisateur et par article
df = df.groupby(["user_id", "article_id"]).sum().reset_index()

# Création des scores
df["rating"] = (df["duration"] / df["estimated_reading_time"]).clip(
    upper=1, lower=0
) + 1
df.drop(columns=["duration", "estimated_reading_time"], inplace=True)

# Ajout des 0 pour les utilisateurs qui n'ont pas noté d'article
users = df["user_id"].unique()
articles = df["article_id"].unique()
all_users = np.repeat(users, len(articles))
all_articles = np.tile(articles, len(users))
all_ratings = np.zeros(len(all_users))
df_all = pd.DataFrame(
    {
        "user_id": all_users,
        "article_id": all_articles,
        "rating": all_ratings,
        "timestamp": 0,
    }
)
df = pd.concat([df, df_all], axis=0)

# Save and display
## Create output folder
output_folder = os.path.join(
    pwd,
    "results",
    "recommendation_datasets",
)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
## Save to csv
df.to_csv(os.path.join(output_folder, "dataset_2.csv"), index=False, sep="\t")
## Display
display(Markdown("#### Dataset 2 : *Based on reading session*"))
display(df)
## Display some information
display(Markdown("- **Number of users**: {}".format(len(df["user_id"].unique()))))
display(Markdown("- **Number of articles**: {}".format(len(df["article_id"].unique()))))
display(
    Markdown(
        "- **Percentage of positive ratings**: {}%".format(
            round(df["rating"] > 0).mean() * 100, 2
        )
    )
)

#### Read the article

Finally, in order to obtain what will certainly be our most extensive dataset, we'll create a dataset by applying the following evaluation strategy:
- **2** if the user has read and liked the article,
- **1** if the user has read the article,
- **0** otherwise.

In [ ]:
# -------------------------- CREATE SURPRISE DATASET ------------------------- #

# Importation des données
df = pd.read_csv(
    os.path.join(
        pwd,
        "data",
        "processed",
        "recommendation",
        "user_article.csv",
    ),
    index_col=0,
)

# Mise en forme des données
df = df[["user_id", "article_id", "liked", "opened"]].dropna()
df.rename(columns={"opened": "timestamp"}, inplace=True)
df["timestamp"] = pd.to_datetime(df["timestamp"]).astype(int) // 10**9
df["article_id"] = df["article_id"].astype(int)
df["rating"] = df["liked"].astype(int) + 1
df.drop(columns=["liked"], inplace=True)
df = df[["user_id", "article_id", "rating", "timestamp"]]

# Ajout des 0 pour les utilisateurs qui n'ont pas noté d'article
users = df["user_id"].unique()
articles = df["article_id"].unique()
all_users = np.repeat(users, len(articles))
all_articles = np.tile(articles, len(users))
all_ratings = np.zeros(len(all_users))
all_timestamps = np.zeros(len(all_users))
df_all = pd.DataFrame(
    {
        "user_id": all_users,
        "article_id": all_articles,
        "rating": all_ratings,
        "timestamp": all_timestamps,
    }
)
df = pd.concat([df, df_all], axis=0)

# Save and display
## Create output folder
output_folder = os.path.join(
    pwd,
    "results",
    "recommendation_datasets",
)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
## Save to csv
df.to_csv(os.path.join(output_folder, "dataset_3.csv"), index=False, sep="\t")
## Display
display(Markdown("#### Dataset 3 : *Based on user article*"))
display(df)
## Display some information
display(Markdown("- **Number of users**: {}".format(len(df["user_id"].unique()))))
display(Markdown("- **Number of articles**: {}".format(len(df["article_id"].unique()))))
display(
    Markdown(
        "- **Percentage of positive ratings**: {}%".format(
            round(df["rating"] > 0).mean() * 100, 2
        )
    )
)